In [8]:
import pandas as pd
from sklearn import metrics

In [24]:
dataset = 'cifar110'
model = 'vgg8'
method = 'complete'
pct_missing = 0.
rounds = ['round1', 'round2']
predictions = [
    'logs/{}/prediction_{}_{}.csv'.format(rnd, dataset, model)
    for rnd in rounds]
evaluation = 'logs/evaluation_{}_{}.csv'.format(dataset, model)
print(predictions)

['logs/round1/prediction_cifar110_vgg8.csv', 'logs/round2/prediction_cifar110_vgg8.csv']


In [29]:
dataset = 'cifar110'
model = 'vgg8'
method = 'pn'
pct_missing = 0.5
rounds = ['round1']
# rounds = ['round1', 'round2', 'round3']
predictions = [
    'logs/{}/prediction_{}_{}_{}_{}.csv'.format(rnd, dataset, model, method, pct_missing)
    for rnd in rounds]
evaluation = 'logs/evaluation_{}_{}.csv'.format(dataset, model)
print(predictions)

['logs/round1/prediction_cifar110_vgg8_pn_0.5.csv']


In [30]:
def evaluate(y_pred, y_true):
    acc = metrics.accuracy_score(y_true, y_pred)
    prec = metrics.precision_score(y_true, y_pred, average='macro')
    rec = metrics.recall_score(y_true, y_pred, average='macro')
    f1 = metrics.f1_score(y_true, y_pred, average='macro')
    
    return acc, prec, rec, f1

In [31]:
records = []
for fcsv in predictions:
    df = pd.read_csv(fcsv)
    train = df[df['split']=='train']
    (acc, prec, rec, f1) = evaluate(train['y_pred'],
                                          train['y_true'])
    record = dict(method=method,
                 pct_missing=pct_missing,
                 split='train',
                 label='true',
                 acc=acc,
                 prec=prec,
                 rec=rec,
                 f1=f1)
    records.append(record)
    (acc_lab, prec_lab, rec_lab, f1_lab) = evaluate(train['y_pred'],
                                                    train['y_label'])
    record_lab = dict(method=method,
                 pct_missing=pct_missing,
                 split='train',
                 label='label',
                 acc=acc_lab,
                 prec=prec_lab,
                 rec=rec_lab,
                 f1=f1_lab)
    records.append(record_lab)
    
    test = df[df['split']=='test']
    (acc_val, prec_val, rec_val, f1_val) = evaluate(test['y_pred'], test['y_true'])
    record_val = dict(method=method,
                 pct_missing=pct_missing,
                 split='test',
                 label='true',
                 acc=acc_val,
                 prec=prec_val,
                 rec=rec_val,
                 f1=f1_val)
    records.append(record_val)

In [32]:
dfeval = pd.DataFrame(records)
dfeval.to_csv(evaluation, index=False, mode='a')